# Feast Quick Start
This is a quick example to demonstrate:
- Register a feature set on Feast
- Ingest features into Feast
- Retrieve the ingested features from Feast
- Update a feature

In [1]:
import feast
import numpy as np
import pandas as pd
from datetime import datetime
from feast.serving.ServingService_pb2 import GetOnlineFeaturesRequest
from feast.types.Value_pb2 import Value as Value
from feast.client import Client
from feast.feature_set import FeatureSet

First, instantiate the client.
Feast endpoints can be set via the following environmental variables: `FEAST_CORE_URL`, `FEAST_SERVING_URL`.
Alternatively, they can also be passed in explicitly as follows:
    
`client = feast.Client(core_url=core:6565, serving_url=online-serving:6566)`

In [2]:
client = feast.Client()

## Register a feature set

Let's create and register our first feature set. Below is an example of a basic customer transactions feature set that has been exported to YAML:
```
name: customer_transactions
kind: feature_set
entities:
- name: customer_id
  valueType: INT64
features:
- name: daily_transactions
  valueType: FLOAT
- name: total_transactions
  valueType: FLOAT
maxAge: 3600s    
```

In [3]:
cust_trans_fs = FeatureSet.from_yaml("../features/cust_trans_fs.yaml")

In [4]:
client.apply(cust_trans_fs)

Feature set updated/created: "customer_transactions:1".


## Ingest features into Feast
The dataframe below contains the features and entities of the above feature set.

In [5]:
offset = 10000
nr_of_customers = 5
customer_df = pd.DataFrame(
    {
        "datetime": [datetime.utcnow() for _ in range(nr_of_customers)],
        "customer_id": [offset + inc for inc in range(nr_of_customers)],
        "daily_transactions": [np.random.uniform(0, 10) for _ in range(nr_of_customers)],
        "total_transactions": [np.random.uniform(100, 200) for _ in range(nr_of_customers)],
    }
)
customer_df

,datetime,customer_id,daily_transactions,total_transactions
0,2019-11-26 12:03:47.320634,10000,5.178112,110.670651
1,2019-11-26 12:03:47.320644,10001,0.268114,195.393913
2,2019-11-26 12:03:47.320651,10002,1.486614,136.929052
3,2019-11-26 12:03:47.320658,10003,9.676433,166.022999
4,2019-11-26 12:03:47.320665,10004,5.928573,165.687951


In [6]:
client.ingest(cust_trans_fs, dataframe=customer_df)

  0%|          | 0/5 [00:00<?, ?rows/s]8rows/s]


Ingested 5 rows into customer_transactions:1


## Retrieve the ingested features from Feast
Once the features are ingested, they can be retrieved from the Feast store for model training or prediction purposes.

In [7]:
client.get_online_features(
    entity_rows=[
        GetOnlineFeaturesRequest.EntityRow(
            fields={
                "customer_id": Value(
                    int64_val=10001
                )
            }
        )
    ],
    feature_ids=[
        "customer_transactions:1:daily_transactions",
        "customer_transactions:1:total_transactions",
    ],
)

field_values {
  fields {
    key: "customer_id"
    value {
      int64_val: 10001
    }
  }
  fields {
    key: "customer_transactions:1:daily_transactions"
    value {
      float_val: 0.2681143283843994
    }
  }
  fields {
    key: "customer_transactions:1:total_transactions"
    value {
      float_val: 195.39390563964844
    }
  }
}

## Update a feature
We can also update feature set which has been ingested.

In [8]:
cust_trans_fs_updated = FeatureSet.from_yaml("../features/cust_trans_fs_updated.yaml")

In [9]:
client.apply(cust_trans_fs_updated)

Feature set updated/created: "customer_transactions:2".


In [10]:
discounts = [np.random.uniform(0, 10) for _ in range(nr_of_customers)]
customer_df_updated = customer_df.assign(discounts=discounts)
customer_df_updated

,datetime,customer_id,daily_transactions,total_transactions,discounts
0,2019-11-26 12:03:47.320634,10000,5.178112,110.670651,8.389938
1,2019-11-26 12:03:47.320644,10001,0.268114,195.393913,0.430047
2,2019-11-26 12:03:47.320651,10002,1.486614,136.929052,7.408917
3,2019-11-26 12:03:47.320658,10003,9.676433,166.022999,1.192721
4,2019-11-26 12:03:47.320665,10004,5.928573,165.687951,2.051037


In [11]:
client.ingest(cust_trans_fs_updated, dataframe=customer_df_updated)

  0%|          | 0/5 [00:00<?, ?rows/s]6rows/s]


Ingested 5 rows into customer_transactions:2


In [12]:
client.get_online_features(
    entity_rows=[
        GetOnlineFeaturesRequest.EntityRow(
            fields={
                "customer_id": Value(
                    int64_val=10001
                )
            }
        )
    ],
    feature_ids=[
        "customer_transactions:2:daily_transactions",
        "customer_transactions:2:total_transactions",
        "customer_transactions:2:discounts",
    ],
)

field_values {
  fields {
    key: "customer_id"
    value {
      int64_val: 10001
    }
  }
  fields {
    key: "customer_transactions:2:daily_transactions"
    value {
      float_val: 0.2681143283843994
    }
  }
  fields {
    key: "customer_transactions:2:discounts"
    value {
      float_val: 0.4300469756126404
    }
  }
  fields {
    key: "customer_transactions:2:total_transactions"
    value {
      float_val: 195.39390563964844
    }
  }
}